# Exploring Survey Definitions

The purpose of this notebook is to demonstrate how to load the parameters of the survey definitions and explore how to use them.

In [ ]:
from os import path, getcwd
from sys import path as pythonpath
pythonpath.append(path.join(getcwd(), '..'))
import config_utils
import regions
from astropy import units as u 
from astropy_healpix import HEALPix
from mw_plot import MWSkyMap
import matplotlib.pyplot as plt

Alternative designs for the Roman Galactic Plane Survey are described in the file config/rgps_survey_definitions.json.  This can be loaded into a Python dictionary using the config_utils provided.   

Each survey design is described as a Python dictionary, with entries for the different optical elements to be used to observe different regions.  Multiple regions can be specified for each optical element.

In [ ]:
survey_def_file = path.join(getcwd(), '..', 'config', 'rgps_survey_definitions.json')
survey_def = config_utils.read_config(survey_def_file)
survey_def

For the purposes of evaluating survey maps, metrics etc, this package uses CelestialRegion objects, which provide a range of functionality.  Rather than have to re-create these objects from the original survey design every time, the conversion to CelestialRegions has already been made and stored in the file config/rgps_survey_definition_regions.json.  
From here, they can be loaded directly as follows.

In [ ]:
# First we need to load the overall parameters of the simulation, which provides the common definitions for the 
# HEALpixel resolution used throughout the simulation and fixed attributes of the Roman mission such as the available filters
sim_config = config_utils.read_config(path.join(getcwd(), '..', 'config', 'sim_config.json'))

# Load the regions defined for each survey description, and summarize them
survey_regions_file = path.join(getcwd(), '..', 'config', 'rgps_survey_regions.json')
survey_regions = regions.load_regions_from_file(sim_config, survey_regions_file)

for name, survey_params in survey_regions.items():
    print('\nSurvey strategy: ' + name)
    for optic, region_set in survey_params.items():
        print(optic, region_set)

Having loaded each survey definition as a set of CelestialRegions, we can now make use of various visualization tools.  

The following example is used to plot the map of the region to be observed in one filter for one particular strategy.

In [ ]:
# Select a strategy and filter to plot 
optic = 'F213' 
survey_name = 'Option_1a'

# Create a plot of the Galactic Plane
mw1 = MWSkyMap(projection='aitoff', grayscale=False, grid='galactic', background='infrared', figsize=(16,10))
mw1.title = survey_name + ', in ' + optic
proj = HEALPix(nside=64, order='ring', frame='icrs')

# Select the list of regions identified for the selected survey strategy and filter 
region_set = survey_regions[survey_name][optic]

# Each CelestialRegion object in the region_set already has an array of the HEALpixels included within it, so we 
# use this array to highlight the survey footprint as an overlay on the plot
for r in region_set:
    s = proj.healpix_to_skycoord(list(r.pixels))
    mw1.scatter(s.ra.deg*u.deg, s.dec.deg*u.deg, c="r", s=5, alpha=0.4)